# Rheological Flow around Confined Cylinder

Results published:
[Kikker, Kummer, Oberlack: A fully coupled high-order discontinuous Galerkin solver for viscoelastic fluid flow](https://onlinelibrary.wiley.com/doi/10.1002/fld.4950), 
see also at [tubiblio](http://tubiblio.ulb.tu-darmstadt.de/125045/).

In [36]:
#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


Databases loaded: 
Capacity: 4
Count: 1



In [37]:
using BoSSS.Application.Rheology;

In [38]:
BoSSSshell.WorkflowMgm.Init("ConfinedCylinder_ConvergenceStudy");

Project name is set to 'ConfinedCylinder_ConvergenceStudy'.


In [39]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,Password,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\flori\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,"[ C:\ == , D:\ == ]",,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\dc1\userspace\kummer\cluster,False,FDY-WindowsHPC,dotnet,,[ \\dc1\userspace\kummer == ],FDY\kummer,,DC2,<null>,Normal,True,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,W:\bosss_deploy,False,LbI,dotnet,,[ ],fk69umer,<null>,lcluster8.hrz.tu-darmstadt.de,,,,C:\Users\flori\.ssh\id_rsa,<null>,/home/fk69umer/bosss_deploy,project01287,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\fk69umer\bosss_deploy,False,Lb2-specialPrj,dotnet,,[ X:\fk69umer == /work/scratch/fk69umer ],fk69umer,<null>,lcluster16.hrz.tu-darmstadt.de,,,,C:\Users\flori\.ssh\id_rsa,"[ #SLURM -p test24, #SLURM -C avx512 ]",/work/scratch/fk69umer/bosss_deploy,special00006,<null>,False


In [5]:
var myBatch = ExecutionQueues[3];

In [6]:
var myDb = myBatch.CreateOrOpenCompatibleDatabase("Rheology_ConfinedCylinder");

Opening existing database 'X:\fk69umer\Rheology_ConfinedCylinder'.


In [7]:
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

## Parameter set for Convergence Study

In [8]:
int[] pOrder = new int[] {1, 2, 3, 4};
int numberGrids = 4;

In [9]:
double[] WeissenbergS = new double[] { 0.0, 0.2 };

## Init grids and save to database

In [10]:
IGridInfo[] grids = new IGridInfo[numberGrids];
//GridCommons grid;
for (int k = 0; k < numberGrids; k++) {

//bosssGrid.Name        = "confined_cylinder";

Func<double[], string> edgeTagFunc = delegate (double[] X) {
    double x = X[0];
    double y = X[1];
    //  if (!C.FixedStreamwisePeriodicBC)
    //  {
        if (Math.Abs(x - (-15)) < 1.0e-10)
            return "Velocity_inlet";
        if (Math.Abs(x - (15)) < 1.0e-10)
            return "Pressure_Outlet";
    //  }
    //  if (Math.Abs(y - (-2)) < 1.0e-10)
    //      return "Freeslip";
                        
        if (Math.Abs(y - (0)) < 1.0e-10)
            return "Freeslip";
                        
        if (Math.Abs(y - (+2)) < 1.0e-10)
            return "Wall_top";
                        
    //  if (-1.0 < y && y < 1.0 && -1.0 < x  && x < 1.0)
    //    return "Wall_cylinder";
                        
        if (0.0 < y && y < 1.0 && -1.0 < x  && x < 1.0)
            return "Wall_cylinder";
                        
        throw new ArgumentOutOfRangeException("at x = " + x + "and y = " +y);
    };

    
    grids[k] = BoSSSshell.WorkflowMgm.ImportGrid(@"Cylinder_GRIDS\mesh_karman_OriginalBox_MEDIUM_"+k+"_half.msh", EdgeTagFunc:edgeTagFunc);
}


Grid Edge Tags changed.
An equivalent grid (4ce4b9bc-cce4-4c8e-a749-ed79d65caca2) is already present in the database -- the grid will not be saved.

Grid Edge Tags changed.
An equivalent grid (557173ad-f0c7-42ce-8bff-acdda333a95d) is already present in the database -- the grid will not be saved.

Grid Edge Tags changed.
An equivalent grid (e245a313-0f98-4039-9a52-1581e12a76ec) is already present in the database -- the grid will not be saved.

Grid Edge Tags changed.
An equivalent grid (1a36165d-47ac-4248-aad4-8a2cfa8c80c4) is already present in the database -- the grid will not be saved.


In [11]:
grids

#0: { Guid = 4ce4b9bc-cce4-4c8e-a749-ed79d65caca2; Name = ; Cell Count = 320; Dim = 2 }
#1: { Guid = 557173ad-f0c7-42ce-8bff-acdda333a95d; Name = ; Cell Count = 1208; Dim = 2 }
#2: { Guid = e245a313-0f98-4039-9a52-1581e12a76ec; Name = ; Cell Count = 4252; Dim = 2 }
#3: { Guid = 1a36165d-47ac-4248-aad4-8a2cfa8c80c4; Name = ; Cell Count = 14712; Dim = 2 }


## Setup of control objects for a solver runs

In [12]:
using BoSSS.Solution.AdvancedSolvers;

In [13]:
// Code for Initial conditions
class MyInitCode {

static string nl = System.Environment.NewLine;
public static string InitCode(double Reynolds, double Weissenberg, double beta) => 
"static class InitialValues { " + nl +
" " + nl +
"    static double u0                   = 1.5; // 0.375;// 0.66;// 3 / 2;   " + nl +
"    static double h                    = 4; " + nl +
$"    public static double beta          = {beta}; " + nl +
$"    static double Reynolds             = {Reynolds}; " + nl +
$"    static double Weissenberg          = {Weissenberg};  " + nl +
"    " + nl +
"    public static double VelocityXfunction(double[] X) { " + nl +
"        return u0 * (1  - (X[1] *  X[1])/h); " + nl +
"    } " + nl +
"    " + nl +
"    public static double VelocityYfunction(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"    " + nl +
"    public static double Pressurefunction(double[] X) { " + nl +
"        return u0 * 0.5 * Reynolds * (35 - X[0]); " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXXfunction_wi02(double[] X) { " + nl +
"        const double Weissenberg = 0.2; " + nl +
"        return 2 * Weissenberg * (1 - beta) * u0 * (-2 / h) * X[1] * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXXfunction_wi00(double[] X) { " + nl +
"        const double Weissenberg = 0.0; " + nl +
"        return 2 * Weissenberg * (1 - beta) * u0 * (-2 / h) * X[1] * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXYfunction(double[] X) { " + nl +
"        return (1 - beta) * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressYYfunction(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"}  " + nl;

}

In [14]:
//MyInitCode.InitCode(1,2)

In [15]:
List<RheologyControl> Controls = new List<RheologyControl>();

In [32]:
Controls.Clear();
double _beta = 0.59;
double _Reynolds = 1.0;
foreach(double Weissenberg in WeissenbergS) {
foreach(int degree in pOrder) {
int elemInd = 0;
foreach(var grd in grids) {

    //Database
    RheologyControl C = new RheologyControl();
    C.savetodb        = true;
    C.DbPath          = myDb.Path;
    C.ProjectName     = "Cylinder";
    C.SetGrid(grd);
    C.TracingNamespaces = "BoSSS,ilPSP";

    //SolverChooser
    C.NonLinearSolver.MaxSolverIterations  = 100;
    C.NonLinearSolver.MinSolverIterations  = 3;
    C.NonLinearSolver.ConvergenceCriterion = 1E-6;
    C.NonLinearSolver.SolverCode           = NonLinearSolverCode.Newton;
    C.LinearSolver.SolverCode = LinearSolverCode.classic_pardiso;

    //Timestepping
    C.TimesteppingMode   = AppControl._TimesteppingMode.Steady;
    C.Timestepper_Scheme = RheologyControl.TimesteppingScheme.ImplicitEuler;
    C.ObjectiveParam     = 1.0;

    //Configuration Shock capturing and body forces
    //C.UsePerssonSensor       = false;
    //C.SensorLimit            = 1e-4;
    C.AdaptiveMeshRefinement = false;
    C.RefinementLevel        = 10;
    //C.UseArtificialDiffusion = false;
    C.Bodyforces             = true;

    //Physical parameters
    C.beta                 = _beta;
    C.Reynolds             = _Reynolds;
    C.Weissenberg          = Weissenberg; //aim Weissenberg number!
    //C.RaiseWeissenberg     = false;
    //C.WeissenbergIncrement = 0.0;

    //Penalties
    C.ViscousPenaltyScaling = 1;
    C.Penalty2              = 1;
    C.Penalty1[0]           = 0.0;
    C.Penalty1[1]           = 0.0;
    C.PresPenalty2          = 1;  
    C.PresPenalty1[0]       = 0.0;
    C.PresPenalty1[1]       = 0.0;
    C.alpha                 = 1;  
    C.StressPenalty         = 1.0;

    //Create Fields
    C.SetDGdegree(degree);

    //Set initial values
    C.InitialValues.Add("VelocityX", new Formula("InitialValues.VelocityXfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.InitialValues.Add("VelocityY", new Formula("InitialValues.VelocityYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    if(Weissenberg == 0.0)
        C.InitialValues.Add("StressXX", new Formula("InitialValues.StressXXfunction_wi00", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else if(Weissenberg == 0.2)
        C.InitialValues.Add("StressXX", new Formula("InitialValues.StressXXfunction_wi02", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else
        throw new NotImplementedException("Missing Weissenberg = " + Weissenberg);    
    C.InitialValues.Add("StressXY", new Formula("InitialValues.StressXYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.InitialValues.Add("StressYY", new Formula("InitialValues.StressYYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));

    //Set Boundary Conditions
    //C.AddBoundaryValue("Wall_top", "VelocityX", Wall);
    //C.AddBoundaryValue("Wall_top", "VelocityY", Wall);
    //C.AddBoundaryValue("Wall_cylinder", "VelocityX", Wall);
    //C.AddBoundaryValue("Wall_cylinder", "VelocityY", Wall);

    C.AddBoundaryValue("Velocity_inlet", "VelocityX", new Formula("InitialValues.VelocityXfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.AddBoundaryValue("Velocity_inlet", "VelocityY", new Formula("InitialValues.VelocityYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    if(Weissenberg == 0.0)
        C.AddBoundaryValue("Velocity_inlet", "StressXX", new Formula("InitialValues.StressXXfunction_wi00", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else if(Weissenberg == 0.2)
        C.AddBoundaryValue("Velocity_inlet", "StressXX", new Formula("InitialValues.StressXXfunction_wi02", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else
        throw new NotImplementedException("Missing Weissenberg = " + Weissenberg);
    C.AddBoundaryValue("Velocity_inlet", "StressXY", new Formula("InitialValues.StressXYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.AddBoundaryValue("Velocity_inlet", "StressYY", new Formula("InitialValues.StressYYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta))); 
            
    //Save Session and next...
    C.SessionName = $"ConfinedCylinder_ConvergenceStudy_p{degree}_meshNo{elemInd}_Wi{Weissenberg}_half";
    Controls.Add(C);
    Console.WriteLine("Created control: " + C.SessionName);
    elemInd++;
}
}
}

Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo1_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo2_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo3_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo1_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo2_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo3_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo1_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo2_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo3_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p4_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p4_meshNo1_Wi0_half
Create

## Launch Jobs

In [33]:
Controls.Count

32

In [34]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    //oneJob.ExecutionTime    = "24:00:00";
    oneJob.Activate(myBatch);
}

Deploying job ConfinedCylinder_ConvergenceStudy_p1_meshNo0_Wi0_half ... 
Deploying executables and additional files ...
Deployment directory: W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_185746


IOException  copy of file 'C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Release\net5.0\ilPSP.dll' --> 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_185746\ilPSP.dll' : The request could not be performed because of an I/O device error. : 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_185746\ilPSP.dll'


Retrying 1 of 10 (waiting for some time before) ...


IOException  copy of file 'C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Release\net5.0\ilPSP.dll' --> 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_185746\ilPSP.dll' : The request could not be performed because of an I/O device error. : 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_185746\ilPSP.dll'


Retrying 2 of 10 (waiting for some time before) ...


...

IOException  copy of file 'C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Release\net5.0\System.Configuration.ConfigurationManager.dll' --> 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_190900\System.Configuration.ConfigurationManager.dll' : The request could not be performed because of an I/O device error. : 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_190900\System.Configuration.ConfigurationManager.dll'


Retrying 9 of 10 (waiting for some time before) ...


IOException  copy of file 'C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\bin\Release\net5.0\System.Configuration.ConfigurationManager.dll' --> 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_190900\System.Configuration.ConfigurationManager.dll' : The request could not be performed because of an I/O device error. : 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_190900\System.Configuration.ConfigurationManager.dll'


Retrying 10 of 10 (waiting for some time before) ...


Unhandled exception: System.IO.IOException: The request could not be performed because of an I/O device error. : 'W:\bosss_deploy\ConfinedCylinder_ConvergenceStudy-RheologySolver2021Jun22_190900\System.Configuration.ConfigurationManager.dll'
   at BoSSS.Application.BoSSSpad.MetaJobMgrIO.RetryIOop(Func`2 op, String Message, Boolean SurpressException) in C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\BatchProcessorClient.cs:line 394
   at BoSSS.Application.BoSSSpad.Job.DeployExecuteables() in C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\Job.cs:line 1519
   at BoSSS.Application.BoSSSpad.Job.Activate(BatchProcessorClient bpc, Boolean DeleteOldDeploymentsAndSessions) in C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\BoSSSpad\Job.cs:line 1212
   at Submission#37.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAs

In [35]:
BoSSSshell.WorkflowMgm.AllJobs

#0: ConfinedCylinder_ConvergenceStudy_p1_meshNo0_Wi0_half: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	ConfinedCylinder_ConvergenceStudy_p1_meshNo0_Wi0_half: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#1: ConfinedCylinder_ConvergenceStudy_p1_meshNo1_Wi0_half: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	ConfinedCylinder_ConvergenceStudy_p1_meshNo1_Wi0_half: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)
#2: ConfinedCylinder_ConvergenceStudy_p1_meshNo2_Wi0_half: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu-darmstadt.de, Slurm account: special00006)	ConfinedCylinder_ConvergenceStudy_p1_meshNo2_Wi0_half: FailedOrCanceled (SlurmClient Lb2-specialPrj : fk69umer@lcluster16.hrz.tu

In [20]:
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value).Where(job => job.Status == JobStatus.FailedOrCanceled).ToArray();

In [21]:
suspects

(empty)

In [ ]:
// wait for all jobs to finish (up to 5 days, check every 5 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*5), PollingIntervallSeconds:(60*5));

In [23]:
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects

(empty)

In [24]:
//foreach(Job j in suspects) {
//    j.DeleteOldDeploymentsAndSessions(); 
//}

In [25]:
var active = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value).Where(job => job.Status == JobStatus.InProgress).ToArray();

In [26]:
//active

In [27]:
//active[0].LatestDeployment.DeploymentDirectory;

In [28]:
//active[0].Stdout;